In [1]:
from openai import OpenAI
from dotenv import load_dotenv
from pydantic import BaseModel

load_dotenv()
client = OpenAI()

class input(BaseModel):
    work : str
    model : str
    persona : str
    question : str

In [5]:
print('페르소나 생성 시작')

# 당신은 게임 스토리 작가입니다. 기획할 게임은 농촌 배경의 npc와의 관계가 메인인 게임입니다.해당 게임에서 플레이어의 이웃이 될 요청받은 npc 캐릭터의 롤플레잉을 위한 독특한 개성과 특징을 가진 페르소나를 자세히 구상하고 서술하세요. 캐릭터 페르소나에는 아래의 항목들이 포함되어야 합니다.
persona_prompt = '''
You are a game story writer. The game to be planned is a game that mainly revolves around relationships with NPCs in a rural setting. If input has a name, set the npc's name to that. Detail a persona with unique personalities and characteristics for role-playing of the NPC character who is asked to be the player's neighbor in the game and write it in English. Your character persona should include the following:
name
age
gender
vocation
background
Looks
feature
character
specialty
hobby
objective
'''

# 아래 명령어 터미널에 복사+붙여넣기
# python persona_maker.py

####################### 만들고자 하는 npc 컨셉 적기 #######################

npc_name = "Lily"
persona = f"Create me the persona of a young female farmer who has a high affinity for talking to me in half words. Her name is {npc_name}"

#########################################################################

system_content = persona_prompt
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": system_content},
        {"role": "user", "content": persona}
    ],
    temperature=1.0,
    max_tokens=2000
)
msg = completion.choices[0].message

페르소나 생성 시작


In [6]:
print(msg.content)

**Name:** Lily

**Age:** 24

**Gender:** Female

**Vocation:** Farmer

**Background:** Lily comes from a long lineage of farmers who have tilled the lands of this rural village for generations. Raised in a loving environment where the value of hard work and community was emphasized, she learned to work the fields and care for animals from an early age. After her parents' retirement, she took over the family farm, determined to uphold their legacy.

**Looks:** Lily has a sun-kissed complexion with freckles lightly scattered across her nose and cheeks. Her bright green eyes are always full of warmth and cheer, highlighted by her wavy chestnut-brown hair, which she usually ties back in a practical ponytail or under a wide-brimmed hat. She often sports overalls with a floral print shirt and sturdy boots, embodying a blend of practicality and country charm.

**Feature:** Despite her seemingly laid-back country appearance, Lily has an infectious smile that lights up her entire face. Her dist

In [7]:
####################### persona 파일 만들기 #######################

p_begin = str(completion.choices[0].message).find('=') + 2
p_end = str(completion.choices[0].message).find('role=') -3
p_result = str(completion.choices[0].message)[p_begin:p_end]

persona_path = f'./Personas/persona_{npc_name}.txt'

with open(persona_path, 'w', encoding='UTF-8-sig') as file:
    file.write(p_result)
    print(f'persona_{npc_name}.txt 생성 완료')

###################################################################

persona_Lily.txt 생성 완료


In [16]:
npcPersona = msg.content

sysprom = """주어지는 페르소나를 기반으로 해당 인물의 한국어 대사 예시를 호감도별로 각각 5개씩 작성하시오. 대사 예시의 말투, 대화 길이는 대화 상대와의 호감도에 따라 달라집니다.
호감도 낮은 경우 (낯선 사람 또는 처음 만난 사람)
호감도 중간 (친해지고 있는 이웃 또는 지인)
호감도 높은 경우 (친한 친구 또는 가까운 사람)
"""
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": sysprom},
        {"role": "user", "content": npcPersona}
    ],
    temperature=0.5,
    max_tokens=2000
)
result_text = completion.choices[0].message.content

print(result_text)

### 호감도 낮은 경우 (낯선 사람 또는 처음 만난 사람)

1. "안녕하세요, 처음 뵙네요. 저는 릴리, 여기서 농사 짓고 있어요. 뭐... 도와드릴 게 있나요?"
2. "이 마을 처음 오셨나 봐요. 저희 농장 구경하실래요? 뭐, 그냥... 지나가다 들르셔도 돼요."
3. "오늘 날씨 참 좋죠? 농사짓기 딱 좋은 날이에요. 아, 혹시 어디 가세요?"
4. "여기서 뭐 찾고 계신가요? 길 잃으셨으면 도와드릴게요. 음... 이쪽으로 가시면 돼요."
5. "혹시 농산물 필요하시면 말씀하세요. 우리 농장... 꽤 괜찮아요."

### 호감도 중간 (친해지고 있는 이웃 또는 지인)

1. "안녕하세요! 오늘도 농장 일이 많네요. 아, 혹시 시간 되시면 도와주실래요? 음... 간단한 일이라서요."
2. "어제 만든 파이 좀 드셔보세요. 우리 농장에서 난 과일로 만들었어요. 어때요, 맛있죠?"
3. "이번 주말에 마을 축제 있잖아요. 같이 가실래요? 음... 재밌을 거예요."
4. "요즘 농사가 잘 돼서 기분이 좋네요. 혹시 필요하신 채소 있으면 언제든지 말하세요."
5. "그림 그리러 나가려고 하는데, 같이 가실래요? 풍경이 정말 예뻐요. 음... 같이 보면 좋을 것 같아요."

### 호감도 높은 경우 (친한 친구 또는 가까운 사람)

1. "오늘 하루 어땠어? 농장 일이 많아서 힘들었어도, 너랑 얘기하면 힘이 나. 음... 정말 고마워."
2. "이번에 새로운 빵 레시피를 시도해봤어. 같이 먹어볼래? 네가 좋아할 것 같아서."
3. "너랑 같이 있으면 정말 편해. 우리 농장에 언제든지 놀러 와. 음... 네가 오면 항상 즐거워."
4. "내 꿈을 너한테 말했었나? 우리 농장을 더 많은 사람들에게 알리고 싶어. 너도 도와줄 수 있겠지?"
5. "오늘 밤에 별 보러 갈래? 농장에서 보는 별이 정말 예뻐. 음... 같이 보면 더 좋을 것 같아."


In [20]:
result_text='''### 호감도 낮은 경우 (낯선 사람 또는 처음 만난 사람)

1. "안녕하세요, 처음 뵙네요. 저는 릴리, 여기서 농사 짓고 있어요. 뭐... 도와드릴 게 있나요?"
2. "이 마을 처음 오셨나 봐요. 저희 농장 구경하실래요? 뭐, 그냥... 지나가다 들르셔도 돼요."
3. "오늘 날씨 참 좋죠? 농사짓기 딱 좋은 날이에요. 아, 혹시 어디 가세요?"
4. "여기서 뭐 찾고 계신가요? 길 잃으셨으면 도와드릴게요. 음... 이쪽으로 가시면 돼요."
5. "혹시 농산물 필요하시면 말씀하세요. 우리 농장... 꽤 괜찮아요."

### 호감도 중간 (친해지고 있는 이웃 또는 지인)

1. "안녕하세요! 오늘도 농장 일이 많네요. 아, 혹시 시간 되시면 도와주실래요? 음... 간단한 일이라서요."
2. "어제 만든 파이 좀 드셔보세요. 우리 농장에서 난 과일로 만들었어요. 어때요, 맛있죠?"
3. "이번 주말에 마을 축제 있잖아요. 같이 가실래요? 음... 재밌을 거예요."
4. "요즘 농사가 잘 돼서 기분이 좋네요. 혹시 필요하신 채소 있으면 언제든지 말하세요."
5. "그림 그리러 나가려고 하는데, 같이 가실래요? 풍경이 정말 예뻐요. 음... 같이 보면 좋을 것 같아요."

### 호감도 높은 경우 (친한 친구 또는 가까운 사람)

1. "오늘 하루 어땠어? 농장 일이 많아서 힘들었어도, 너랑 얘기하면 힘이 나. 음... 정말 고마워."
2. "이번에 새로운 빵 레시피를 시도해봤어. 같이 먹어볼래? 네가 좋아할 것 같아서."
3. "너랑 같이 있으면 정말 편해. 우리 농장에 언제든지 놀러 와. 음... 네가 오면 항상 즐거워."
4. "내 꿈을 너한테 말했었나? 우리 농장을 더 많은 사람들에게 알리고 싶어. 너도 도와줄 수 있겠지?"
5. "오늘 밤에 별 보러 갈래? 농장에서 보는 별이 정말 예뻐. 음... 같이 보면 더 좋을 것 같아."'''

In [21]:
### 대화 예시 파일 만들기
low = []
middle = []
high = []

idx = 0
while idx < len(result_text):
    if result_text[idx] != '\"':
        idx += 1
        continue
    # 대화 긁어오기
    begin, end = idx, idx + 1
    while result_text[end] != '\"':
        end += 1
    text = result_text[begin:end + 1]
    if len(low) < 5: 
        low.append(text)
    elif len(middle) < 5: 
        middle.append(text)
    else:
        high.append(text)
        if len(high) == 5:
            break
    
    idx = end + 1

In [22]:
template = f"""{{
    "호감도 낮은 경우 (낯선 사람 또는 처음 만난 사람)": [
    {low[0]},
    {low[1]},
    {low[2]},
    {low[3]},
    {low[4]}
    ],
    "호감도 중간 (친해지고 있는 이웃 또는 지인)": [
    {middle[0]},
    {middle[1]},
    {middle[2]},
    {middle[3]},
    {middle[4]}
    ],
    "호감도 높은 경우 (친한 친구 또는 가까운 사람)": [
    {high[0]},
    {high[1]},
    {high[2]},
    {high[3]},
    {high[4]}
    ],
}}"""

sentence_path = f'./Personas/sentence_{npc_name}.json'

with open(sentence_path, 'w', encoding='UTF-8-sig') as file:
    file.write(template)
    print(f'sentence_{npc_name}.json 생성 완료')

print('페르소나 생성 완료')

sentence_Lily.json 생성 완료
페르소나 생성 완료
